In [1]:
import sys; sys.path.append('../..')
import random
import pyzx as zx
import os
import pickle
import time
%load_ext autoreload
%autoreload 2 

In [2]:
random.seed(1344)
c = zx.Circuit.load('../../circuits/feyn_bench/qasm/gf2^4_mult.qasm').to_basic_gates()
c = zx.optimize.basic_optimization(c.split_phase_gates())
g = c.to_graph()
# g = zx.generate.cliffordT(qubits=8, depth=1000, p_t=0.4)
# c = zx.Circuit.from_graph(g)
print(c.stats())

Circuit  on 12 qubits with 207 gates.
        96 is the T-count
        111 Cliffords among which 
        99 2-qubit gates (96 CNOT, 3 other) and
        8 Hadamard gates.


In [3]:
gnf = g.copy()
zx.simplify.clifford_simp(gnf)
zx.topt.prepare_gadget_form(gnf)
gnf.normalize()
zx.draw(gnf)

In [4]:
gpg = gnf.copy()
zx.topt.eliminate_cliffords(gpg)
gpg.normalize()
zx.draw(gpg)

pivot_simp: 49. 16. 12. 13. 4. 4. 3.  7 iterations
iteration:  1


In [5]:
gfg = gpg.copy()
zx.simplify.gadget_simp(gfg)
gfg.normalize()
zx.draw(gfg, labels=True)

gadget_simp: 9.  1 iterations


In [6]:
for neigh in gfg.neighbors(32):
    if len(gfg.neighbors(neigh)) == 1:
        print(neigh)

In [7]:
zx.simplify.clifford_simp(gfg)
gfg.normalize()
zx.draw(gfg, labels=True, scale=50)

In [19]:
def theroutine():
    zx.topt.prepare_gadget_form(gfg)
    zx.topt.eliminate_cliffords(gfg)
    zx.simplify.gadget_simp(gfg)

for i in range(1,5):
    theroutine()

zx.simplify.clifford_simp(gfg)

pivot_simp: 2. 1.  2 iterations
iteration:  1
pivot_simp: 2. 1. 1.  3 iterations
iteration:  1
pivot_simp: 2. 1.  2 iterations
iteration:  1
pivot_simp: 1. 1. 1.  3 iterations
iteration:  1


1

In [20]:
gfg.normalize()
zx.draw(gfg)

In [14]:
zx.editor.edit(gfg, scale=50)

ZXEditorWidget(graph_buttons='{"spider": {"active": false, "text": "fuse spiders", "tooltip": "Fuses connected…

Output()

ZXEditorWidget(graph_buttons='{"spider": {"active": false, "text": "fuse spiders", "tooltip": "Fuses connected…

In [ ]:
def get_gadget_roots(g):
    roots = []
    for v in g.vertex_set():
        for n in g.neighbors(v):
            if len(g.neighbors(n)) == 1:
                roots.append(v)
                break
#         if len(g.neighbors(v)) == 2 and g.phases()[v].numerator <= 1:
#             roots.append(v)
    return roots

def check_same_neighbors(g, first, second):
    print(set(g.neighbors(first)),set(g.neighbors(second)))
    if set(g.neighbors(first)) == set(g.neighbors(second)):
        return True
    return False

def check_all_gadgets(g, rootset):
    rc = rootset[:]
    while len(rc) > 0:
        candidate = rc.pop()
        similar = []
        for other in rc:
            if check_same_neighbors(g, candidate, other):
                similar.append(other)
        if len(similar) > 0:
            print("found similars for candidate ",candidate,similar)

check_all_gadgets(gfg, get_gadget_roots(gfg))

In [33]:
zx.draw(gfg, labels=True)

In [29]:
zx.simplify.lcomp_simp(g)
zx.simplify.pivot_simp(g)

lcomp_simp: 149. 100. 80. 50. 36. 34. 24. 22. 17. 12. 11. 8. 9. 7. 8. 8. 5. 7. 7. 5. 4. 5. 3. 3. 4. 2. 1. 2. 2. 1.  30 iterations
pivot_simp: 4. 4.  2 iterations


2

In [30]:
zx.draw(g)

In [31]:
zx.simplify.gadget_simp(g)

gadget_simp: 8.  1 iterations


1

In [32]:
zx.draw(g)

In [15]:
c1 = zx.extract_circuit(gfg.copy())

TypeError: 'float' object cannot be interpreted as an integer

In [9]:
c1.stats()

'Circuit  on 12 qubits with 535 gates.\n        68 is the T-count\n        467 Cliffords among which \n        287 2-qubit gates (11 CNOT, 276 other) and\n        172 Hadamard gates.'

In [10]:
gfull = g.copy()
zx.simplify.full_reduce(gfull)
zx.extract_circuit(gfull.copy()).stats()

'Circuit  on 12 qubits with 488 gates.\n        68 is the T-count\n        420 Cliffords among which \n        256 2-qubit gates (23 CNOT, 233 other) and\n        156 Hadamard gates.'

In [11]:
ctodd = zx.Circuit.from_graph(g.copy())
ctodd = zx.optimize.basic_optimization(ctodd.split_phase_gates())
ctodd = zx.optimize.phase_block_optimize(ctodd.split_phase_gates())

In [12]:
ctodd.stats()

'Circuit  on 12 qubits with 300 gates.\n        57 is the T-count\n        243 Cliffords among which \n        225 2-qubit gates (192 CNOT, 33 other) and\n        8 Hadamard gates.'